# **Inferencing Fine-tuned Llama2 model**

## **Installing Dependancies**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q langchain transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 107.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 102.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.7 MB/s eta 0:00:00


In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 104.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00


In [ ]:
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain


from transformers import AutoModel
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

import json
import textwrap

## **Loading Lora Adapter and Base LLM model**

## **PEFT model**

### **1. For Llama-2-7b-chat-hf model**

In [ ]:
from huggingface_hub import login
login()

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
model_name='meta-llama/Llama-2-7b-chat-hf'
new_model='/content/drive/MyDrive/My_personality_new_emotion/resultschat01/llama2chat/final_checkpoint'
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map='auto',
)
model = PeftModel.from_pretrained(base_model, new_model)
model_mer = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### **2. For Llama-2-7b-hf**

In [ ]:
from huggingface_hub import login
login()

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
model_name='meta-llama/Llama-2-7b-hf'
new_model='/content/drive/MyDrive/My_personality_new_emotion/resultschat01/llama2chat/final_checkpoint'
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map='auto',
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

##**3.For Llama-2-13b-chat-hf Model**

In [ ]:
from huggingface_hub import login
login()

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
model_name='meta-llama/Llama-2-13b-chat-hf'
new_model='/content/drive/MyDrive/My_personality_new_emotion/resultschat02/llama2chat/final_checkpoint'
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    return_dict=True,
    torch_dtype=torch.float16,
    device_map='auto',
)
model = PeftModel.from_pretrained(base_model, new_model)
# model_mer = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## **Original model Llama 2**

## **`1.Llama-2-7b-chat-hf**

In [ ]:
from huggingface_hub import login
login()

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
model_name='meta-llama/Llama-2-7b-chat-hf'
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map='auto',
)


# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## **2.Llama-2-13b-chat-hf**

In [ ]:
from huggingface_hub import login
login()

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
model_name='meta-llama/Llama-2-13b-chat-hf'
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map='auto',
)


# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

## **Save merged Model**

In [ ]:
model_mer.save_pretrained("/content/drive/MyDrive/My_personality_new_emotion/Merged_model04")
tokenizer.save_pretrained("/content/drive/MyDrive/My_personality_new_emotion/Merged_model04")

('/content/drive/MyDrive/My_personality_new_emotion/Merged_model04/tokenizer_config.json',
 '/content/drive/MyDrive/My_personality_new_emotion/Merged_model04/special_tokens_map.json',
 '/content/drive/MyDrive/My_personality_new_emotion/Merged_model04/tokenizer.json')

## **Load the quantized Fine-tuned Llama2 model**

In [ ]:
model_path='/content/drive/MyDrive/My_personality_new_emotion/Merged_model04'
model = AutoModelForCausalLM.from_pretrained(model_path,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             load_in_4bit=True,
                                             bnb_4bit_quant_type="nf4",
                                             bnb_4bit_compute_dtype=torch.float16)

tokenizer = AutoTokenizer.from_pretrained(model_path)

## ***Inferencing Fine tuned LLM with Langchain***

## **Create pipeline for inferencing**

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model_mer,
                tokenizer= tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


## **Create Prompting for inferencing**

In [ ]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

In [ ]:
def get_prompt(instruction, citation=None):
    prompt_template =  B_INST + instruction + E_INST

    if citation:
        prompt_template += f"\n\nCitation: {citation}"  # Insert citation here

    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")

def generate(text, citation=None):
    prompt = get_prompt(text, citation=citation)
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(**inputs,
                                 max_length=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs

def parse_text(text):
    wrapped_text = textwrap.fill(text, width=100)
    print(wrapped_text + '\n\n')

In [ ]:
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.7,'max_length': 256, 'top_k' :50})


In [ ]:
# instruct = 'You are an expert in the Big Five personality traits, which are:Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism.Analyze the following dialogue and determine which SINGLE trait from the Big Five best represents the speaker. Your analysis should be strictly based on the content of the dialogue, and you should avoid making assumptions not present in the text.'
instruct="You are an expert in the Big Five personality traits.\n These traits are: Openness: Characterized by an individual's imaginative and insightful nature. People high in openness are usually curious, open to new experiences, and appreciate art and beauty.Conscientiousness: Represents how organized and dependable an individual is. Those high in conscientiousness are typically methodical, goal-oriented, and reliable.Extraversion: Denotes an individual's level of outgoingness and sociability. Extraverts are usually talkative, energetic, and enjoy being around others.Agreeableness: Reflects how kind, affectionate, and cooperative an individual is. People high in agreeableness are usually compassionate and get along well with others.Neuroticism: Indicates emotional instability. Individuals high in neuroticism tend to experience mood swings, anxiety, irritability, and sadness.Based on the provided dialogue, determine which SINGLE trait from the Big Five best represents the speaker. Your analysis should be strictly based on the content of the dialogue, and you should avoid making assumptions not present in the text. Ensure your response is accurate, unbiased, and respectful. Remember, this is a classification task, so provide only one trait as the output.Let's think step by step.Give proper short reason for your answer"
sys_template = B_SYS + instruct + E_SYS
prompt_template = B_INST + sys_template + """\nINPUT: {text} \nhere's the response:""" + E_INST

In [ ]:
print(prompt_template)

[INST]<<SYS>>
You are an expert in the Big Five personality traits.
 These traits are: Openness: Characterized by an individual's imaginative and insightful nature. People high in openness are usually curious, open to new experiences, and appreciate art and beauty.Conscientiousness: Represents how organized and dependable an individual is. Those high in conscientiousness are typically methodical, goal-oriented, and reliable.Extraversion: Denotes an individual's level of outgoingness and sociability. Extraverts are usually talkative, energetic, and enjoy being around others.Agreeableness: Reflects how kind, affectionate, and cooperative an individual is. People high in agreeableness are usually compassionate and get along well with others.Neuroticism: Indicates emotional instability. Individuals high in neuroticism tend to experience mood swings, anxiety, irritability, and sadness.Based on the provided dialogue, determine which SINGLE trait from the Big Five best represents the speaker.

In [ ]:
prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm, verbose = True)

In [ ]:
text = "Im so happy for the Saints!!! Now time to sober up and look forward to the Olympics"

## **Getting Response**

In [ ]:
response = llm_chain.run(text)
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are an expert in the Big Five personality traits.
 These traits are: Openness: Characterized by an individual's imaginative and insightful nature. People high in openness are usually curious, open to new experiences, and appreciate art and beauty.Conscientiousness: Represents how organized and dependable an individual is. Those high in conscientiousness are typically methodical, goal-oriented, and reliable.Extraversion: Denotes an individual's level of outgoingness and sociability. Extraverts are usually talkative, energetic, and enjoy being around others.Agreeableness: Reflects how kind, affectionate, and cooperative an individual is. People high in agreeableness are usually compassionate and get along well with others.Neuroticism: Indicates emotional instability. Individuals high in neuroticism tend to experience mood swings, anxiety, irritability, and sadness.Based on the provided dialogue, determine which

## **Generating Response With Test file**

In [ ]:
import pandas as pd

# Load the CSV file into a pandas DataFrame
df = pd.read_csv('/content/modified_file_01.csv')

# Select only the first 10 samples
df=df.iloc[:100]

# Define a function to get the prediction using the external model/function
def get_prediction(status):
    return llm_chain.run(status)

# Apply the function to get predictions
df['predicted'] = df['STATUS'].apply(get_prediction)

# Convert the DataFrame to a list of dictionaries
results = []
for _, row in df.iterrows():
    result = {
        "predicted_output": row['predicted'],
        "true_label": row['personality_final']
    }
    results.append(result)

# Save the results to a JSON file
import json

with open('output_final_llama2-7b-PEFT03.json', 'w') as f:
    json.dump(results, f)



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are an expert in the Big Five personality traits.
 These traits are: Openness: Characterized by an individual's imaginative and insightful nature. People high in openness are usually curious, open to new experiences, and appreciate art and beauty.Conscientiousness: Represents how organized and dependable an individual is. Those high in conscientiousness are typically methodical, goal-oriented, and reliable.Extraversion: Denotes an individual's level of outgoingness and sociability. Extraverts are usually talkative, energetic, and enjoy being around others.Agreeableness: Reflects how kind, affectionate, and cooperative an individual is. People high in agreeableness are usually compassionate and get along well with others.Neuroticism: Indicates emotional instability. Individuals high in neuroticism tend to experience mood swings, anxiety, irritability, and sadness.Based on the provided dialogue, determine which

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are an expert in the Big Five personality traits.
 These traits are: Openness: Characterized by an individual's imaginative and insightful nature. People high in openness are usually curious, open to new experiences, and appreciate art and beauty.Conscientiousness: Represents how organized and dependable an individual is. Those high in conscientiousness are typically methodical, goal-oriented, and reliable.Extraversion: Denotes an individual's level of outgoingness and sociability. Extraverts are usually talkative, energetic, and enjoy being around others.Agreeableness: Reflects how kind, affectionate, and cooperative an individual is. People high in agreeableness are usually compassionate and get along well with others.Neuroticism: Indicates emotional instability. Individuals high in neuroticism tend to experience mood swings, anxiety, irritability, and sadness.Based on the provided dialog

## **Modification in Json File**

In [ ]:
import json

# Read the JSON file
with open('/content/output_final_llama2-7b-PEFT03.json', 'r') as file:
    data = json.load(file)

# List of Big Five personality traits
traits = ["Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Neuroticism"]

# Check if data is a list
if isinstance(data, list):
    for entry in data:
        predicted_output = entry.get('predicted_output', None)

        # If predicted_output exists and is a string
        if predicted_output and isinstance(predicted_output, str):
            for trait in traits:
                if trait.lower() in predicted_output.lower():
                    entry['predicted_output'] = trait
                    break
else:
    print("The structure of the JSON data does not match the expected format.")

# Save the modified data back to JSON
with open('modified_output_final_llama2-7b-PEFT03.json', 'w') as file:
    json.dump(data, file, indent=4)


### **Calculate F1 score for single Label Test file**

In [ ]:
import json
from sklearn.metrics import f1_score, classification_report

# Load JSON data
with open('/content/modified_output_final3.json', 'r') as file:
    data = json.load(file)

# Extract predicted and true labels
predicted_labels = [entry['predicted_output'] for entry in data]
true_labels = [entry['true_label'] for entry in data]

# Compute F1 score
f1_macro = f1_score(true_labels, predicted_labels, average='macro')
f1_micro = f1_score(true_labels, predicted_labels, average='micro')
f1_weighted = f1_score(true_labels, predicted_labels, average='weighted')

print(f"Macro F1 Score: {f1_macro}")
print(f"Micro F1 Score: {f1_micro}")
print(f"Weighted F1 Score: {f1_weighted}")

# If you want a detailed classification report:
print(classification_report(true_labels, predicted_labels))


### **Calculate F1 score for multi label Test file**

In [ ]:
import json

# Load JSON data
with open('/content/modified_output_final_llama2-7b-PEFT03.json', 'r') as file:
    data = json.load(file)

# Extract predicted and true labels
predicted_labels = [entry['predicted_output'] for entry in data]

# Split the comma-separated true labels into lists
true_labels_lists = [entry['true_label'].split(", ") for entry in data]

TP = 0
FP = 0

# For each pair of predicted label and list of true labels
for predicted, true_list in zip(predicted_labels, true_labels_lists):
    if predicted in true_list:
        TP += 1
    else:
        FP += 1

# Compute F1 score using the custom approach
f1 = (2 * TP) / (2 * TP + FP)

print(f"F1 Score: {f1:.4f}")


F1 Score: 0.5714


In [ ]:
import json
from sklearn.metrics import f1_score

# Load JSON data
with open('/content/modified_output_final3.json', 'r') as file:
    data = json.load(file)

# Extract predicted and true labels
predicted_labels = [entry['predicted_output'] for entry in data]

# Split the comma-separated true labels into lists
true_labels_lists = [entry['true_label'].split(", ") for entry in data]

# Create lists for true and predicted binary values
y_true_bin = []
y_pred_bin = []

# Populate the binary lists
for predicted, true_list in zip(predicted_labels, true_labels_lists):
    for true_label in true_list:
        y_true_bin.append(true_label)
        y_pred_bin.append(predicted)

# Convert the lists to binary format (1 for match, 0 for no match)
y_true_bin = [1 if label == pred else 0 for label, pred in zip(y_true_bin, y_pred_bin)]
y_pred_bin = [1] * len(y_true_bin)  # Since we are comparing against itself

# Compute F1 score using sklearn's function
f1 = f1_score(y_true_bin, y_pred_bin)

print(f"F1 Score: {f1:.4f}")


F1 Score: 0.2960


## **Test file recreation**

In [ ]:
import pandas as pd

# Load the csv file into a DataFrame
df = pd.read_csv('/content/drive/MyDrive/My_personality_new_emotion/test.csv')

# Function to get top two personality traits
def top_two_traits(row):
    # Extracting the scores
    scores = {
        'Extraversion': row['Extraversion'],
        'Neuroticism': row['Neuroticism'],
        'Agreeableness': row['Agreeableness'],
        'Conscientiousness': row['Conscientiousness'],
        'Openness': row['Openness']
    }

    # Sorting the dictionary by scores in descending order and taking the first two keys (traits)
    top_two = sorted(scores, key=scores.get, reverse=True)[:2]

    # Handling cases where two (or more) traits have the same score by checking the scores of the top two traits
    if scores[top_two[0]] == scores[top_two[1]]:
        return ', '.join(sorted(top_two))  # Sort alphabetically for consistent output
    else:
        return ', '.join(top_two)

# Apply the function to each row of the DataFrame to get the top two traits
df['personality_final'] = df.apply(top_two_traits, axis=1)

# Save the modified DataFrame to a new csv file
df.to_csv('modified_file.csv', index=False)


### **F1 score Testing**

In [ ]:
import json
from sklearn.metrics import f1_score

# Load JSON data
with open('/content/modified_output_final_llama2-7b.json', 'r') as file:
    data = json.load(file)

# Extract predicted and true labels
predicted_labels = [entry['predicted_output'] for entry in data]
true_labels_lists = [entry['true_label'].split(", ") for entry in data]

# Create a binary list of predictions: 1 if predicted label matches any of the true labels, 0 otherwise
y_pred_binary = [1 if predicted in true_list else 0 for predicted, true_list in zip(predicted_labels, true_labels_lists)]
y_true_binary = [1] * len(y_pred_binary)  # All ones since we want a match for all entries

# Compute F1 score
f1 = f1_score(y_true_binary, y_pred_binary)

print(f"F1 Score: {f1:.4f}")


F1 Score: 0.6441
